In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_pickle(r'data\04_fct\fct_demographic_offers_and_transactions.pkl')
df.head()

,age,income,days_as_member,gender_F,gender_M,offer_id,offer_viewed,offer_completed,viewed_before_completion,difficulty,reward,duration_hrs,email,mobile,social,web,is_bogo,is_discount,total_transactions,total_transaction_amount
customer_id,,,,,,,,,,,,,,,,,,,,
0610b486422d4921ae7d2bf64640c50b,55,112000,376,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,1,0,5,5,168,1,1,0,1,1,0,1,23.22
0610b486422d4921ae7d2bf64640c50b,55,112000,376,1,0,3f207df678b143eea3cee63160fa8bed,0,0,0,0,0,96,1,1,0,1,0,0,0,0.00
78afa995795e4d85b5d9ceeca43f5fef,75,100000,443,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,1,5,5,168,1,1,0,1,1,0,1,19.89
78afa995795e4d85b5d9ceeca43f5fef,75,100000,443,1,0,5a8bc65990b245e5a138643cd4eb9837,1,0,0,0,0,72,1,1,1,0,0,0,0,0.00
78afa995795e4d85b5d9ceeca43f5fef,75,100000,443,1,0,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,10,10,168,1,1,1,0,1,0,1,21.72


age                         float64
income                      float64
days_as_member              float64
gender_F                       int8
gender_M                       int8
offer_id                     object
offer_viewed                   int8
offer_completed                int8
viewed_before_completion       int8
difficulty                     int8
reward                         int8
duration_hrs                  int16
email                          int8
mobile                         int8
social                         int8
web                            int8
is_bogo                        int8
is_discount                    int8
total_transactions             int8
total_transaction_amount    float64
cluster                       int32
dtype: object

In [3]:
# Standardize demographic data
scaler = StandardScaler()
df[['age', 'income', 'days_as_member']] = scaler.fit_transform(df[['age', 'income', 'days_as_member']])

In [6]:
from sklearn.cluster import KMeans

# Select demographic data
X = df[['age', 'income', 'days_as_member']]

# Perform K-means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
df['cluster'] = kmeans.fit_predict(X)

# Profile clusters by calculating the mean only for numeric columns
numeric_cols = df.select_dtypes(include=['number']).columns
cluster_profile = df.groupby('cluster')[numeric_cols].mean()

In [8]:
# Group by clusters and analyze offer responses
pivot = df.pivot_table(index='cluster', 
                       values=['offer_viewed', 'offer_completed'], 
                       columns=['is_bogo', 'is_discount', 'mobile', 'social', 'web'],
                       aggfunc='mean')
pivot

offer_completed                                               \
is_bogo                   0                                            1   
is_discount               0              1                             0   
mobile                    1              0         1                   1   
social                    0    1         0         0         1         0   
web                       1    0         1         1         1         1   
cluster                                                                    
0                       0.0  0.0  0.585326  0.736088  0.932334  0.757353   
1                       0.0  0.0  0.676429  0.724845  0.806642  0.726885   
2                       0.0  0.0  0.355182  0.450614  0.629306  0.528293   

                                offer_viewed                                \
is_bogo                                    0                                 
is_discount                                0                   1             
mobile                                     1                   0         1   
social              1                      0         1         0         0   
web                 0         1            1         0         1         1   
cluster                                                                      
0            0.582248  0.650807     0.612383  0.912638  0.353669  0.628162   
1            0.730047  0.704111     0.620866  0.844413  0.431743  0.625179   
2            0.397099  0.441150     0.454049  0.957149  0.292737  0.454711   

                                                     
is_bogo                       1                      
is_discount                   0                      
mobile                        1                      
social              1         0         1            
web                 1         1         0         1  
cluster                                              
0            0.967024  0.611928  0.921824  0.974511  
1            0.969854  0.606923  0.826761  0.970363  
2            0.967785  0.480308  0.953309  0.961974